In [ ]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [ ]:
"""!pip install datasets
!pip install transformers
!pip install sentence_transformers
!pip install peft
!pip install huggingface_hub
!pip install --upgrade accelerate
!pip install --upgrade bitsandbytes
!pip install --upgrade torchmetrics
!pip install --upgrade deepspeed
"""

In [ ]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
#from chat_template_utils import get_json_schema
#from llama_cpp import Llama
from utils import *
from peft import *
import datasets
#import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
#from hqq.engine.hf import HQQModelForCausalLM
#from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
#from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)


In [ ]:
with open('result.json','r',encoding='utf-8') as file:
    
    j = file.read()
    j = json.loads(j)
    print(j.keys())
    

In [ ]:
#j['messages'][1]['from']
#datasets.Dataset.from_json('result.json',field="messages")
df = pd.read_json('result.json')


info_df = pd.json_normalize(df['messages'])[['from','text']]

#joined = info_df['from']+': '+info_df['text']
info_df = info_df[~(info_df['text']=='')].dropna().reset_index(drop=True)
#info_df[~(info_df['from']=='')]

info_df

In [ ]:
c = info_df['text'].str.contains("\[",na=True)
x = info_df.loc[c,:].reset_index()
x

In [ ]:
print(x.loc[0,'text'])

In [ ]:
c1 = info_df['from']=='مسعود شرقی'
c2 = info_df['from'] == 'مجید'
info_df[~( c1 | c2 )]

In [ ]:
def get_one_msg(x):
    text = x['text']
    if not isinstance(text,str):
        #print(text[0])
        return pd.NA
    
    return  { 
     "role":x['from'],
     "content": text
    }
    
data = info_df.apply(get_one_msg,axis=1)
data = data.dropna()
data

In [ ]:
def to_chat(t):
    s=''
    for x in t[1:]:
        s += f"{x['role']}: {x['content']}\n"
    
    return s
    


windows_size = 20
train_dataset = []
for i in range(0,len(data)-windows_size,windows_size-5):
    #d = [{"role":"system","content":"زبان شما فارسی است."}]
    #d.extend( data[i:i+windows_size].tolist() )
    d = data[i:i+windows_size].tolist()
    train_dataset.append(d)

print(len(train_dataset))    
for i in range(len(train_dataset)):
    train_dataset[i]= to_chat(train_dataset[i])
    
train_dataset


In [ ]:
from GraphRag.utils import  completion
#user_prompt = """چت بین دو نفر در متن زیر امده است شما باید پیام های مرتبط به هم را در یک پیام بنویسد اگر پیامی نامرتبط بود ابتدا سعی کنید با ترکیب ان با پیام های قبلی یک پیام مرتبط و با مفهوم بنویسید اگر نتوانستید ان پیام را حذف کنید و یک مکالمه با معنا و مرتب شده بنویسید پاسخ پیام ها اغلب در زیر آن امده است ولی نه لزومن 
#""" می خواهیم از داده های چت بین دو نفر استفاده کنیم و یک مدل زیانی آموزش بدهیم. مهم این است که داد ها تمیز و مرتب شوند شما باید هر پیام و جواب آن را بیابید و آن ها را مرتب کنید. در اغلب موارد جواب پیام در زیر آن امده است ولی لزوما این گونه نیست پیام های نامرتب را نیز حذف کنید و به فارسی بنویسید

example="""### مثال ###
چت:
[{'role': 'system', 'content': 'زبان شما فارسی است.'}, {'role': 'مسعود شرقی', 'content': 'اره دیگه'}, {'role': 'مسعود شرقی', 'content': 'امتحانات تموم شد؟'}, {'role': 'مجید', 'content': 'خیلی عجیبه همچین چیزی ندیده بودم تا حالا :/'}, {'role': 'مجید', 'content': 'نه 3 شمبه تموم میشه'}, {'role': 'مجید', 'content': 'من خودم یه نرم افزار دارم اخرین بازدیدو مثه همین مخفی میکنه'}, {'role': 'مجید', 'content': 'اما وقتی یه پیام میفرستی انلاین میشه'}, {'role': 'مجید', 'content': 'بدیش همینه'}, {'role': 'مجید', 'content': 'از تو خیلی خوفه'}, {'role': 'مجید', 'content': 'اهان داشتم اینو میگفتم یه ساعت 2 ملیونو نیمو فروخته 100 هزار تومن'}, {'role': 'مجید', 'content': 'احتمال 90 درصد مجازیه'}, {'role': 'مجید', 'content': 'اینو ببین حال کن'}, {'role': 'مجید', 'content': '😁✋'}, {'role': 'مجید', 'content': '😃'}, {'role': 'مسعود شرقی', 'content': 'واسه من نشون نمیده\nمیگه اپدیت باید بکنی'}, {'role': 'مجید', 'content': 'مسعود'}, {'role': 'مجید', 'content': 'هسیییییی.'}, {'role': 'مجید', 'content': '؟'}, {'role': 'مسعود شرقی', 'content': 'هئم؟'}, {'role': 'مجید', 'content': 'مسعووووود'}, {'role': 'مجید', 'content': 'امروز می خام برم کیبورد بخرم'}, {'role': 'مجید', 'content': 'چی چی بخرم؟'}, {'role': 'مسعود شرقی', 'content': 'کیبورد من چشه مگه؟'}, {'role': 'مجید', 'content': 'سخته'}, {'role': 'مجید', 'content': 'دکمه ها کوچیکه'}, {'role': 'مجید', 'content': 'سفته'}, {'role': 'مسعود شرقی', 'content': 'اها'}, {'role': 'مسعود شرقی', 'content': 'خوب'}, {'role': 'مسعود شرقی', 'content': 'فک کنم حدود 30 هزار تومن باشه'}, {'role': 'مجید', 'content': 'امروز طالقانی بازه برم؟'}, {'role': 'مسعود شرقی', 'content': 'هر مدلی بخری خوبه.چون کیبورد سیستم خیلی ساده ای داره-بعدشم اینهایی که تعداد دکمه بالایی داره دکمه هاش اصلاااا به کار نمیاد'}, {'role': 'مسعود شرقی', 'content': 'ساده هاشو بخر دیر تر خراب میشه'}, {'role': 'مجید', 'content': 'باشه چه مارکی خوبه؟'}, {'role': 'مسعود شرقی', 'content': 'نمیدونم هر مارکی باشه بگو ضمانت داشته باشه'}, {'role': 'مسعود شرقی', 'content': 'اخه مارکی نیست که هزار مدل مارک اومده'}, {'role': 'مجید', 'content': 'باشه دستت درد نشه'}, {'role': 'مسعود شرقی', 'content': 'اره بازه'}, {'role': 'مجید', 'content': 'مسعووووووووود'}, {'role': 'مجید', 'content': 'نمی تونی یه رباط بسازی که متن ها رو از من بگیره بزاره تو کانال؟'}, {'role': 'مجید', 'content': 'ینی 10 ساله برنامه نویسی می کنیا  😐'}, {'role': 'مجید', 'content': '😂'}, {'role': 'مسعود شرقی', 'content': 'نه حسش نیس'}, {'role': 'مجید', 'content': 'مسعووود'}, {'role': 'مسعود شرقی', 'content': 'هااا'}, {'role': 'مجید', 'content': 'الان داری چیکا می کنی؟'}, {'role': 'مجید', 'content': 'خسه نشی یهو :/'}, {'role': 'مجید', 'content': 'من خیلی بیکارم'}, {'role': 'مجید', 'content': 'بیام پایین؟ 😁'}, {'role': 'مجید', 'content': '🙈🙈🙈🙈'}, {'role': 'مجید', 'content': 'مسعووووووود'}, {'role': 'مجید', 'content': 'من درکت میکنم'}]

خروجی:
مسعود شرقی: اره دیگه
مجید: آره، اصن یه چیز عجیبیه تا حالا همچین چیزی ندیده بودم 😐

مسعود شرقی: امتحانات تموم شد؟
مجید: نه بابا، سه‌شنبه تموم میشه، خستم کردنا 😓

مجید: من خودم یه نرم‌افزار دارم آخرین بازدیدو مثه همین مخفی میکنه، اما وقتی یه پیام می‌فرستی آنلاین میشه، بدیش همینه
مسعود شرقی: دقیقا! کلا این مخفی بازیا آخرش تابلو میشه 😂

مجید: از تو خیلی خوفه
مسعود شرقی: همیشه در خدمتیم داداش 😎

مجید: اهان داشتم اینو می‌گفتم یه ساعت ۲ میلیونو نیمو فروخته ۱۰۰ هزار تومن، احتمال ۹۰ درصد مجازیه
مسعود شرقی: صد در صد مجازیه، این قیمتا خیلی تابلو فیکه

مجید: اینو ببین حال کن 😁✋
مسعود شرقی: ایول! چه باحاله 😃

مسعود شرقی: واسه من نشون نمیده، میگه اپدیت باید بکنی
مجید: جدی؟ برو اپدیت کن، شاید اینجوری اوکی شد

مجید: مسعود
مسعود شرقی: هئم؟

مجید: امروز می‌خوام برم کیبورد بخرم، چی چی بخرم؟
مسعود شرقی: ساده بگیر، دکمه‌ها زیاد نباشه، زود خراب نمیشه

مجید: کیبورد من دکمه‌هاش کوچیکه، سخته، سفته
مسعود شرقی: آره، این خیلی رو مخه. برو یه نرم و راحتشو بگیر، راضی‌تر میشی

مجید: امروز طالقانی بازه برم؟
مسعود شرقی: آره، بازه برو یه سر بزن

مجید: باشه، چه مارکی خوبه؟
مسعود شرقی: هر مارکی باشه فقط بگو ضمانت داشته باشه. هزار تا مدل اومده جدیداً

مجید: مسعووووووووود
مسعود شرقی: بگووو؟

مجید: نمی‌تونی یه ربات بسازی که متن‌ها رو از من بگیره بزاره تو کانال؟ یعنی ۱۰ ساله برنامه‌نویسی می‌کنی 😐😂
مسعود شرقی: حالشو ندارم داداش، این کارا انرژیش بالاس 😂

مجید: الان داری چیکار می‌کنی؟ من خیلی بیکارم، بیام پایین؟ 😁
مسعود شرقی: بیا داداش، دور هم باشیم بیکاری کمتر حس میشه

### مثال ###"""

#c = to_chat(train_dataset[3])
c = train_dataset[8]

user_prompt=\
""" می خواهیم از داده های چت بین دو نفر استفاده کنیم و یک مدل زیانی آموزش بدهیم. مهم این است که داد ها تمیز و مرتب شوند به گونه ای که اگر پیامی به مدل زبانی دادیم جواب با معنا و مفهومی بدهد. برای این کار شما باید پیام های هر شخص را ترکیب نموده و در صورت لزوم چت ها را تغییر دهید و جواب با معنایی را برای هر پیام بنویسد.
### چت ###
{chat}
### چت ###
"""#.format(chat=c,ex=example)

user_prompt2=\
""" می خواهیم یک دیتابیس از پرسش و پاسخ های چت زیر بسازیم. برای این منظور شما باید هر پیام و جواب صحیح ان را بنویسید شما میتوانید چت را تغییر دهید و جواب با معنا را خود بنویسید
### چت ###
{chat}
### چت ###
فرمت خروجی: پیام -> پاسخ به آن پیام این فیلد نباید خالی باشد
""".format(chat=c)


user_prompt3=\
""" زبان شما فارسی غیر رسمی و محاوره ای است. شما باید خود را به جای شخص بگذارید پس جواب ها نباید سوم شخص باشد و برای پیام های زیر پاسخ مناسب کامل و با معنا و مفهموی با دانش خود بنویسید.
{example}
### چت ###
{chat}
### چت ###
فرمت خروجی:اسم شخص: پیام ->اسم شخص: پاسخ به آن پیام این فیلد نباید خالی باشد
""".format(chat=c,example="")

results =  []
for i,c in enumerate(train_dataset):
    try:
        res = completion(user_prompt.format(chat=c),"",
                        #"زبان شما فارسی محاوره ای و غیر رسمی است.",
                        False)
        print(i)
        results.append(res)
    except:
        continue
    
    
    




#print(res)

In [ ]:
import chromadb



In [ ]:
from GraphRag.utils import client,split_list,Embed
from chromadbx import DocumentSHA256Generator
#client.delete_collection('farsi')
farsi_col= client.get_or_create_collection('farsi',
                                                        embedding_function= Embed(),
                                                        metadata={"hnsw:space": "cosine"} # "l2", "ip, "or "cosine"

                                                        )

train_dataset_documents = []
for x in train_dataset:
    train_dataset_documents.append(str(x))
    
len(train_dataset_documents)

sp = split_list(train_dataset_documents,100)
for d in sp:
    ids = DocumentSHA256Generator(documents=d)
    #farsi_col.add(ids,documents=d)
    

In [ ]:
#from fastembed import SparseTextEmbedding
#model = SparseTextEmbedding(model_name="Qdrant/bm25")


In [ ]:
from GraphRag.utils import answer_to_quesion

#question  = "داری چیکار میکونی؟"
#question  = "مسعود: سلام امتحانات کی تموم میشه؟"
#question  = "مسعود کجایی؟"
#question  = "مجید: کی بیام پایین"
#question = "مجید: چطوری مخ بزنم؟"
question = "لوکس بلاگ چن میفروشی؟"


#farsi_col= client.get_or_create_collection('farsi')

reterived = farsi_col.query(query_texts=[question],n_results=1)['documents'][0]
print(len(reterived))
reterived = ''.join(reterived)
#prompt = """شما باید به پیام با توجه به پیام های قبلی پاسخ کامل بلند غیر رسمی و معنا دار بدهید 
sys_p=" شما در حال چت با شخص دیگری هستید. شما باید خود را به جای آن شخص بگذارید و به پیام ها پاسخ دهید. از پیام های قبلی برای پاسخ به پیام استفاده کنید "
prompt =  sys_p+""" 
###
{reterived}
###
با استفاده از پیام های بالا به در یک پیام به پیام زیر به فرمت اسم شخص -> پاسخ

{question}
""".format(question=question,reterived=reterived)

print(prompt)
#completion(prompt,"",False)


In [ ]:
from transformers import Qwen2ForCausalLM,T5ForConditionalGeneration
device = 'cuda'
model_name = 'Qwen/Qwen2-0.5B-Instruct'
#model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

In [ ]:

model = Qwen2ForCausalLM.from_pretrained(model_name #Qwen/Qwen2-0.5B-Instruct
                                         #,attn_implementation="flash_attention_2"
                                         #,torch_dtype=torch.float16
                                         , device_map='auto',cache_dir=cache_dir).to('cuda')

model.eval()
#model.config['use_sliding_window']=True
print(model.config)

model

In [ ]:
tokenized_train_data= tokenizer.apply_chat_template(train_dataset,truncation=True,max_length=512)


In [ ]:
l = []
for i in range(len(tokenized_train_data)):
    l.append(len(tokenized_train_data[i]))
    

l

In [ ]:
l = pd.DataFrame.from_dict(l)
print(l.describe())
l.hist()

In [ ]:
tokenizer.decode( tokenized_train_data[0] )

In [ ]:
model.generation_config

In [ ]:
model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q_proj','v_proj','k_proj',
                                                                                     'o_proj',
                                                                                     'lm_head',
                                                                                     'gate_proj','up_proj','down_proj']))
#model = get_peft_model(model,LoraConfig(task_type=TaskType.CAUSAL_LM,target_modules=['q','v','k']))

model.print_trainable_parameters()
model
for n,p in model.named_parameters():
    if p.requires_grad==False:
        print(n)

In [ ]:
print(tokenizer("<|im_start|>"))

In [ ]:
mydc = DataCollatorForLanguageModeling(tokenizer,False)
def mmf(x):
    y = mydc(x)
    b = 1
    for i in range(len(y['labels'][0])-1,-1,-1  ):
        if y['labels'][0][i]==151644:
                b= i
                break

    for i in range(0,b):
        y['labels'][0][i]=-100
        pass
    return y

y = mmf(tokenized_train_data[0:1])
y



In [ ]:
from transformers import TrainerCallback
"""
class ChangeDatasetCallback(TrainerCallback):
    def __init__(self):
        #super.__init__()
        pass

    def on_epoch_end(self, args, state, control, **kwargs):
        # Move to the next dataset at the end of each epoch
        #self.current_epoch += 1
        #new_dataset = self.datasets[self.current_epoch % len(self.datasets)]
        kwargs['train_dataloader'].dataset = get_train_data()
        #kwargs['eval_dataloader'].dataset = new_dataset['validation']
"""
"""
class custom_dataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.dataset = get_train_data()
        self.len = len(self.dataset)
        pass

    def __getitem__(self, i) :
        if i==self.len-1:
            self.dataset = get_train_data()

        return self.dataset[i]
        pass

    def __len__(self):
        return self.len
        pass
"""
trainer =  Trainer(model,
        train_dataset=tokenized_train_data,
        data_collator=mydc,
        #compute_metrics=compute_metric,
        #callbacks=[ChangeDatasetCallback()],
        tokenizer=tokenizer,
        args=TrainingArguments(
          output_dir=output_dir,
          num_train_epochs=5,
          per_device_train_batch_size=1,
          per_device_eval_batch_size=1,
          logging_steps=300,

          #eval_accumulation_steps =2,


        ))

trainer.train()
model = model.eval()

In [ ]:
model.save_pretrained(output_dir+'/farsi5/')

In [ ]:
d =[
    {"role":"system","content":"زبان شما فارسی است."},
    #{"role":"مجید" ,"content":"خبی"},
    {"role":"مجید" ,"content":"بیکاری؟"},
    
    #{"role":"مجید" ,"content":"ببخشید عاقا میتونم بخشی از سهام لوکس بلاگو بخرم؟"},
    {"role":"مسعود شرقی" ,"content":""},
    
    
    
]
t = tokenizer.apply_chat_template(d,return_tensors='pt')[:,:-2].to('cuda')
#get_one_msg(d)
print(t)
print(tokenizer.decode(t[0]))
print('----------------------------------')
model.eval()
with torch.no_grad():
    o = model.generate(input_ids=t,min_new_tokens=0,max_new_tokens =1000,do_sample=True)#,stop_strings="<asa>?",tokenizer = tokenizer)
    print(tokenizer.decode(o[0]))


In [ ]:
questions = [
"use html to create email"
,"""what is your gender?"""
,"""where are you living?(optional)"""
,"what is your salary expectation?*"
,"Which pronouns should we use to address you? (optional)"
,"how is your German?"
,"do you need visa spunsership?"
,"where are you working right now?"
,"what is your birth date?"
,"what are your hobbies?"
,"password"
,"what projects have you done?"
,"are you a text ai?"
,"what experice do you have in network engineering?"
,"what projects have you done in deep learning?"
,"how did you find us?"]

#model.disable_adapter()

x= [get_one_chat_template(""#person_to_str(person)
                          ,q+"""<input type="text" style="" id="custom">""") for q in questions]
tokenized_questions = tokenizer.apply_chat_template(x)

with torch.no_grad():
    
    for i,x in enumerate(tokenized_questions):
            x = torch.tensor([x]).to('cuda')
            o = model.generate(input_ids=x,max_new_tokens=500,do_sample=False)
            
            print(questions[i])
            #decoded = tokenizer.decode(o[0,x.shape[1]+3:])
            decoded = tokenizer.decode(o[0])
            
            print(decoded)
            print('--------------------------------------------------------')
        
    

In [ ]:
model = model.to('cuda')
def get_answer(decoded):
    try:
                #print("ERROR: ",decoded)

                _s = decoded.find('{')
                _e = decoded.find('}',_s)+1
                #print(decoded[_s:_e])
                answer = json.loads(decoded[_s:_e])['answer']
                return answer
                
    except Exception as e:
                print("ERROR: ",e)
                print(decoded)
                print("---------------------------------")

                
def compute_metric():
    model.eval()
    acc=0
    with torch.no_grad():
        for i in range(len(no_label)):
            x = mydc([no_label[i]])['input_ids'].to('cuda')
            #print(x)
            o = model.generate(input_ids=x,max_new_tokens=20,do_sample=False)
            decoded = tokenizer.decode(o[0,x.shape[1]:])
            #print(decoded)
            #break

            #print(x[0],x[1])
            answer = get_answer(decoded)
            _label = test_dataset.loc[i,'label']
            #_label = person[test_dataset.loc[i,'label']]
            c = answer==_label

            if c:
                acc +=1
            else:
                print("NO_CORRECT:",answer,_label)
                print("---------------------------------")

                pass

    acc/=len(no_label)
    print("acc:",acc)



compute_metric()

In [ ]:
model.eval()
i =11
d = mydc(no_label[i:i+1])['input_ids'].to(device)
d = tokenizer.apply_chat_template( [#{"role":"system","content":""}
                             {"role":"user", "content":"""Your answer has to be in json format like {"answer":""}. Use this content in your answer:\n"""+s}
                            ,{"role":"user", "content":'email'}] , return_tensors='pt').to(device)
#d = tokenizer(['json {"answer":""} what is this html element about? <input type="email">'],return_tensors='pt')['input_ids'].to(device)
#print(d)
with torch.no_grad():

    o = model.generate(input_ids = d,max_length = 512,do_sample=True)
    print(tokenizer.decode(o[0]))



In [ ]:
"""x =Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B-Instruct',cache_dir=cache_dir)
model = PeftModelForCausalLM.from_pretrained(x,output_dir+'saved/')"""

In [ ]:
output_dir